# Retail Sales Insights Dashboard

This notebook focuses on:
- Exploratory Data Analysis (EDA)
- 3 Essential KPIs: Total Revenue, AOV, Category Contribution
- 1 Trend Analysis: Monthly Revenue Trends
- Simple Visualizations and Basic SQL Operations

## 1. Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import sys
import os

from generate_data import generate_sales_data

print("Libraries imported successfully")

In [ ]:
# Data loading and cleaning
def load_and_clean_data(csv_path='sales.csv'):
    try:
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} records from {csv_path}")
    except FileNotFoundError:
        print(f"File not found: {csv_path}")
        return None
    
    print("Starting data cleaning...")
    
    df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
    df = df.dropna(subset=['order_date'])
    
    critical_columns = ['order_id', 'customer_id', 'product', 'category', 'quantity', 'price']
    df = df.dropna(subset=critical_columns)
    
    df = df[(df['quantity'] > 0) & (df['price'] > 0)]
    
    df['revenue'] = df['quantity'] * df['price']
    df['revenue'] = df['revenue'].round(2)
    
    df['month'] = df['order_date'].dt.to_period('M').astype(str)
    
    df['quantity'] = df['quantity'].astype(int)
    df['price'] = df['price'].round(2)
    
    print(f"Data cleaning completed. Final shape: {df.shape}")
    return df

df = load_and_clean_data()
if df is not None:
    print("Data loaded successfully")
else:
    print("Failed to load data")

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Dataset Overview
print("DATASET OVERVIEW")
print("=" * 40)
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Date Range: {df['order_date'].min().date()} to {df['order_date'].max().date()}")
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Data Types and Basic Statistics
print("DATA TYPES:")
print(df.dtypes)
print("\nDESCRIPTIVE STATISTICS:")
df.describe()

In [ ]:
# Data Quality Assessment
print("DATA QUALITY CHECK:")
print("=" * 30)
print("Missing values per column:")
print(df.isnull().sum())

print("\nDataset Summary:")
print(f"Total Records: {len(df):,}")
print(f"Unique Customers: {df['customer_id'].nunique():,}")
print(f"Unique Products: {df['product'].nunique()}")
print(f"Product Categories: {df['category'].nunique()}")
print(f"Date Range: {(df['order_date'].max() - df['order_date'].min()).days} days")

In [ ]:
# Category Distribution Analysis
print("CATEGORY ANALYSIS:")
print("=" * 25)
category_counts = df['category'].value_counts()
print("Order count by category:")
print(category_counts)

print("\nRevenue by category:")
category_revenue = df.groupby('category')['revenue'].sum().sort_values(ascending=False)
for cat, rev in category_revenue.items():
    print(f"{cat}: ${rev:,.2f}")

In [ ]:
# Price and Quantity Analysis
print("PRICE & QUANTITY ANALYSIS:")
print("=" * 35)
print(f"Price Range: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
print(f"Average Price: ${df['price'].mean():.2f}")
print(f"Quantity Range: {df['quantity'].min()} - {df['quantity'].max()}")
print(f"Average Quantity: {df['quantity'].mean():.1f}")

print("\nPrice distribution by category:")
price_by_cat = df.groupby('category')['price'].agg(['min', 'max', 'mean']).round(2)
print(price_by_cat)

## 3. Key Performance Indicators (3 Essential KPIs)

In [ ]:
# KPI calculations
def compute_kpis(df):
    print("Computing Essential KPIs...")
    
    total_revenue = df['revenue'].sum()
    average_order_value = df['revenue'].mean()
    
    total_rev = df['revenue'].sum()
    category_revenue = df.groupby('category')['revenue'].sum().sort_values(ascending=False)
    category_pct = (category_revenue / total_rev * 100).round(1)
    
    category_summary = pd.DataFrame({
        'revenue': category_revenue,
        'percentage': category_pct
    })
    
    monthly_trend = df.groupby('month')['revenue'].sum().sort_index()
    
    kpis = {
        'total_revenue': total_revenue,
        'average_order_value': average_order_value,
        'category_contribution': category_summary,
        'monthly_trend': monthly_trend
    }
    
    print("KPI computation completed")
    return kpis

def display_kpis(kpis):
    print("\n" + "="*40)
    print("KEY PERFORMANCE INDICATORS")
    print("="*40)
    
    print(f"\nTotal Revenue: ${kpis['total_revenue']:,.2f}")
    print(f"Average Order Value: ${kpis['average_order_value']:.2f}")
    
    print(f"\nCategory Contribution:")
    for category, row in kpis['category_contribution'].iterrows():
        print(f"  {category}: ${row['revenue']:,.2f} ({row['percentage']}%)")
    
    print("\n" + "="*40)

kpis = compute_kpis(df)
display_kpis(kpis)

## 4. Trend Analysis (Monthly Revenue Trends)

In [ ]:
# Monthly revenue trends
print("MONTHLY REVENUE TRENDS:")
print("=" * 30)
monthly_trend = kpis['monthly_trend']
for month, revenue in monthly_trend.items():
    print(f"{month}: ${revenue:,.0f}")

if len(monthly_trend) > 1:
    latest = monthly_trend.iloc[-1]
    previous = monthly_trend.iloc[-2]
    growth = ((latest - previous) / previous) * 100
    print(f"\nLatest month growth: {growth:.1f}%")

## 5. Simple Visualizations

In [ ]:
# Monthly Revenue Trend Chart
monthly_data = kpis['monthly_trend']
plt.figure(figsize=(10, 6))
plt.plot(monthly_data.index, monthly_data.values, marker='o')
plt.title('Monthly Revenue Trend')
plt.xlabel('Month')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Category Revenue Chart
category_data = kpis['category_contribution']
plt.figure(figsize=(10, 6))
plt.bar(category_data.index, category_data['revenue'])
plt.title('Revenue by Category')
plt.xlabel('Category')
plt.ylabel('Revenue ($)')
plt.xticks(rotation=45)
plt.show()

## 6. Basic SQL Operations

In [ ]:
# Export to SQLite database
print("Exporting to SQLite database...")

conn = sqlite3.connect('retail.db')
df.to_sql('sales', conn, if_exists='replace', index=False)
print(f"Exported {len(df)} records to database")
conn.close()

In [ ]:
# SQL Query 1: Total Revenue
conn = sqlite3.connect('retail.db')
result = pd.read_sql_query("SELECT SUM(revenue) as total_revenue FROM sales", conn)
print(f"Total Revenue: ${result['total_revenue'].iloc[0]:,.0f}")
conn.close()

In [ ]:
# SQL Query 2: Average Order Value
conn = sqlite3.connect('retail.db')
result = pd.read_sql_query("SELECT AVG(revenue) as average_order_value FROM sales", conn)
print(f"Average Order Value: ${result['average_order_value'].iloc[0]:.2f}")
conn.close()

In [ ]:
# SQL Query 3: Category Contribution
conn = sqlite3.connect('retail.db')
result = pd.read_sql_query("""
SELECT 
    category,
    SUM(revenue) as category_revenue,
    ROUND(SUM(revenue) * 100.0 / (SELECT SUM(revenue) FROM sales), 1) as percentage
FROM sales 
GROUP BY category 
ORDER BY category_revenue DESC
""", conn)
print("Category Contribution:")
for _, row in result.iterrows():
    print(f"{row['category']}: ${row['category_revenue']:,.0f} ({row['percentage']}%)")
conn.close()

In [ ]:
# SQL Query 4: Monthly Revenue Trend
conn = sqlite3.connect('retail.db')
result = pd.read_sql_query("""
SELECT 
    month,
    SUM(revenue) as monthly_revenue
FROM sales
GROUP BY month
ORDER BY month
""", conn)
print("Monthly Revenue Trend:")
for _, row in result.iterrows():
    print(f"{row['month']}: ${row['monthly_revenue']:,.0f}")
conn.close()

## 7. Summary

In [ ]:
# Final summary
print("ANALYSIS SUMMARY:")
print("=" * 25)
print(f"EDA: Analyzed {len(df):,} sales records")
print(f"Total Revenue: ${kpis['total_revenue']:,.2f}")
print(f"AOV: ${kpis['average_order_value']:.2f}")
print(f"Top Category: {kpis['category_contribution'].index[0]}")
print(f"Trend: {len(kpis['monthly_trend'])} months analyzed")

print("\nGenerated Files:")
print("   - retail.db")

print("\nAnalysis completed")